In [1]:
from filterServices import filterEasterEggs
import H5toDataframe as readH5
import outlierServices

# H5 Dateien einlesen
all_dataframes = readH5.convertAllH5Files()

# Easteregg rows entfernen
all_dataframes = filterEasterEggs(all_dataframes)

# Timestamp outlier entfernen
all_dataframes = outlierServices.removeOutliersTurkeysMethod(all_dataframes)

ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
unsupported operand type(s) for -: 'numpy.bytes_' and 'numpy.bytes_'
Ausreißer in d19dde88-83ba-45e6-b23a-e3f1412ffd55.h5:

Ausreißer in 8acbe2e4-b46c-40a2-b635-6df21f699d25.h5:

 - Ersetze Ausreißer an Index 85 (3698) mit 1267205570.0
 - Ersetze Ausreißer an Index 97 (3689) mit 1267209190.5
 - Ersetze Ausreißer an Index 116 (3666.0) mit 1267214922.5
 - Ersetze Ausreißer an Index 161 (3683.0) mit 1267228495.0
 - Ersetze Ausreißer an Index 314 (3609.0) mit 1267255317.0
 - Ersetze Ausreißer an Ind

In [2]:
for name, (df, attrs) in all_dataframes.items():
    print(df.keys())

Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      dtype='object')
Index(['defect_channel', 'distance', 'magnetization', 'timestamp', 'velocity',
       'wall_thickness'],
      

In [5]:
import influxServices

influxServices.upload_dict_with_frames(all_dataframes, 'bucket6')

NewConnectionError: <urllib3.connection.HTTPSConnection object at 0x179e08160>: Failed to establish a new connection: [Errno 61] Connection refused

# Magnetization zu wall_thickness graph

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

def filter_valid_floats(arr):
    filtered_arr = []

    for value in arr:
        try:
            float_value = float(value)
            filtered_arr.append(float_value)
        except ValueError:
            # Wert konnte nicht in float konvertiert werden, wird ignoriert
            pass

    return filtered_arr

# influx_dataframes = filterEasterEggs(influx_dataframes)

# Daten sammeln
magnetization_values = []
wall_thickness_values = []

for name, (df, attrs) in all_dataframes.items():
    if 'magnetization' in df.columns and 'wall_thickness' in df.columns:
        nan_magnetization = df.loc[df['magnetization'].isnull(), 'magnetization']
        nan_wall_thickness = df.loc[df['wall_thickness'].isnull(), 'wall_thickness']

        # if not nan_magnetization.empty:
        #     print(f"NaN in Magnetization in DataFrame {name}: {nan_magnetization}")
        # if not nan_wall_thickness.empty:
        #     print(f"NaN in wall_thickness in DataFrame {name}: {nan_wall_thickness}")

        # Entferne NaN-Werte, bevor du die Werte der Listen hinzufügst
        magnetization_values.extend(df['magnetization'].dropna())
        wall_thickness_values.extend(df['wall_thickness'].dropna())

wall_thickness_values = wall_thickness_values[:-246]

magnetization_values = filter_valid_floats(magnetization_values)
wall_thickness_values = filter_valid_floats(wall_thickness_values)

print(f'Länge Magnetization: {len(magnetization_values)}')
print(f'Länge wall_thickness: {len(wall_thickness_values)}')

# DataFrame erstellen
data = {'magnetization': magnetization_values, 'wall_thickness': wall_thickness_values}
merged_df = pd.DataFrame(data, dtype=np.float64)

# Seaborn-Plot erstellen
sns.scatterplot(x='wall_thickness', y='magnetization', data=merged_df)
plt.title('Magnetization vs. Wall Thickness')
plt.show()